## 서비스 (Servics)

### 1. 서비스란?
- 외부에서 클러스터 내의 **여러개의 Pod에 접근할 수 있는 하나의 IP를 제공**
- 근본적으로는 **로드 밸런서** 역할을 하며 다양한 서비스 제공
- 클러스터 내에서 노드를 옮겨 다니며 **동적으로 변하는 파드들에 고정적인 접근**을 위한 방법
- 파드가 클러스터 안 다른 위치로 옮겨져 IP변하더라도 자동으로 위치가 변한 파드와 통신
- **라벨 셀럭터리를 통한 특정 라벨을 가지고 있는 Pod선택하여 서비스**로 묶음

<hr>



### 2. 서비스 타입
- (1) ClusterIP
    - 기본 서비스 타입으로 클러스터 IP를 이용해서 서비스 연결 파드 접근
- (2) NodePort
    - 서비스 하나에 모든 노드의 지정된 포트 할당 
    - ex) node1:8080, node2:8080 -> 노드에 상관없이 지정된 포트로 파드 접근 가능
- (3) LoadBalancer
    - aws, gcp와 같은 Public cloud service, private cloud 용 로드 밸런서 이용 방법
- (4) ExternalName
    - 클러스터안에서 외부에 접근할 대 주로 사용
    - 서비스를 spec.externalNane필드에 설정한 값과 연결하여 사용

![kubernetes_sevice](img/kubernetes_sevice.PNG)
- nodeport는 cluster-ip 10.110.14.145와 연결되어 있고 node에 접속하여, 3080으로 요청 가능 

<hr>


### 3. Kube-proxy
- 서비스를 만들시, 클러스터 IP나 노드 포트로 접근할 수 있게 실제 조작을 하는 컴포넌트
- 쿠버네티스의 노드마다 실행되어 클러스터 내부 ip로 연결하려는 요청을 적절한 파드로 전달
- Kube-proxy가 네트워크를 관리하는 모드 3가지 userspace, iptables, IPVS 방법 존재
- 기본 userspace(왼쪽) -> 현재 iptables(오른쪽) -> 나중 IPVS

![kubernetes_kube_proxy](img/kubernetes_kube_proxy.png)

- (1) userspace
    - 서비스 클러스터 IP요청 -> iptables 거쳐 kube-proxy 전달 -> 서비스 cluter ip가 적절한 파드연결
- (2) iptables 모드 
    - kube-proxy가 iptables만 관리, 클라이언트 IP 요청 -> iptables이 받아 적절한 파드에 직접 전달
    - 직접전달로 요청 처리 성능이 좋으나, 연결요청 실패시 재시도 x (readneiss probe 헬스체크 결합하여 사용)
- (3) IPVS 모드
    - 커널 공간 동작 및 데이터 구조 해시 테이블로 저장
    - 리눅스 커널에 있는 L4 로드 밸런싱 기술
    - 로드 밸런싱 알고리즘 여러가지 제공 (라운드 로빈… 등)
